# 🏎️ Databricks Notebook Tour: Build F1 Data Pipeline
*Create a complete medallion architecture in 15 minutes*

---

## 🎯 What We'll Build

**Complete Formula 1 Data Lakehouse:**
```
📁 Volume (Raw Files)    →    🥉 Bronze (Raw Tables)    →    🥈 Silver (Clean Tables)    →    🥇 Gold (Analytics)
├── races.csv                 ├── bronze_races              ├── silver_races               ├── gold_driver_standings
├── drivers.csv               ├── bronze_drivers            ├── silver_drivers             └── gold_season_stats  
└── results.csv               └── bronze_results            └── silver_results
```

**🔥 Key Features:**
- ⚡ **Serverless compute** (no cluster management)
- 📁 **Volumes** for file storage (no DBFS)
- 🔄 **COPY INTO** for production-ready ingestion
- 🐍 **Python + SQL** multi-language development
- 📊 **8 tables** across medallion layers

---

## ⚡ Step 1: Serverless Compute Setup

**📌 IMPORTANT:** Make sure you're using **Serverless compute** for this workshop!

### How to Verify Serverless Compute:
1. Look at the top-right of this notebook
2. You should see "Serverless" in the compute dropdown
3. If not, click the dropdown and select "Serverless"

### Why Serverless?
- ✅ **No cluster management** - starts instantly
- ✅ **Auto-scaling** - handles any workload size
- ✅ **Cost efficient** - pay per second of actual usage
- ✅ **Always up-to-date** - latest Databricks runtime

*🎯 Once you see "Serverless" in the compute dropdown, continue to the next cell!*

## 🌟 Step 2: Multi-Language Demo

One of Databricks' superpowers is **seamless multi-language support**. Let's see it in action:

In [ ]:
# Python cell - let's start with some basic info
print("🏎️ Welcome to the F1 Data Pipeline!")
print("=" * 50)

# Get current user and workspace info
current_user = spark.sql("SELECT current_user() as user").collect()[0].user
workspace_id = spark.conf.get("spark.databricks.workspaceUrl", "databricks-workspace")

print(f"👤 Current user: {current_user}")
print(f"🏢 Workspace: {workspace_id}")
print(f"⚡ Compute: Serverless")
print(f"📚 Catalog: main.default")

In [ ]:
# SQL cell - let's check our catalog structure
# Use %sql magic command for SQL in Python notebook
spark.sql("""
  SELECT 
    '🏁 Starting F1 Data Pipeline Build!' as message,
    current_catalog() as current_catalog,
    current_schema() as current_schema,
    current_timestamp() as build_started_at
""").display()

## 📁 Step 3: Create Volume for Data Storage

**Volumes** are the modern way to store files in Databricks. No more DBFS!

### Why Volumes?
- 🔒 **Unity Catalog integration** - full governance
- 🌐 **Cloud-native** - direct cloud storage access  
- 📁 **File system semantics** - works like local folders
- 🔄 **Version control friendly** - easy backup and sync

In [ ]:
# Create our Volume for storing raw F1 data files
# This is where we'll download and store our CSV files

spark.sql("""
CREATE VOLUME IF NOT EXISTS main.default.f1_raw_data
COMMENT 'Raw Formula 1 datasets for workshop - races, drivers, and results'
""")

In [ ]:
# Verify our Volume was created successfully
spark.sql("DESCRIBE VOLUME main.default.f1_raw_data").display()

## 📥 Step 4: Download F1 Data to Volume

Time to get our Formula 1 data! We'll download 3 CSV files from GitHub and store them in our Volume.

**📊 Data Overview:**
- **races.csv** - Race information (circuits, dates, seasons)
- **drivers.csv** - Driver profiles (names, nationalities, birth dates)  
- **results.csv** - Race results (positions, points, lap times)

*📈 Dataset size: ~25,000 race results from 1950-2023*

In [ ]:
import requests
import os

# F1 data source URLs from GitHub
base_url = "https://raw.githubusercontent.com/toUpperCase78/formula1-datasets/master"
files_to_download = {
    "races.csv": f"{base_url}/races.csv",
    "drivers.csv": f"{base_url}/drivers.csv", 
    "results.csv": f"{base_url}/results.csv"
}

# Volume path where we'll store the files
volume_path = "/Volumes/main/default/f1_raw_data"

print("🏎️ Downloading Formula 1 datasets...")
print("=" * 50)

for filename, url in files_to_download.items():
    print(f"📥 Downloading {filename}...")
    
    try:
        # Download the file
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Write to Volume
        file_path = f"{volume_path}/{filename}"
        with open(file_path, 'wb') as f:
            f.write(response.content)
            
        # Check file size
        file_size = len(response.content)
        print(f"   ✅ Downloaded {filename} ({file_size:,} bytes)")
        
    except Exception as e:
        print(f"   ❌ Error downloading {filename}: {str(e)}")

print("\n🎯 Download complete! Ready to build our pipeline.")

In [ ]:
# Let's verify our files are in the Volume
spark.sql("LIST '/Volumes/main/default/f1_raw_data/'").display()

## 🥉 Step 5: Bronze Layer - Raw Data Ingestion

The **Bronze layer** stores raw data exactly as received. We'll use **COPY INTO** for production-ready data ingestion.

### Why COPY INTO?
- 🔄 **Idempotent** - safe to run multiple times
- 📊 **Schema evolution** - handles changing data structures
- 🎯 **Performance** - optimized for bulk loading
- 🔍 **Monitoring** - detailed load statistics

In [ ]:
# Bronze Table 1: Races
spark.sql("""
CREATE OR REPLACE TABLE main.default.bronze_races
USING DELTA
COMMENT 'Bronze layer: Raw F1 race data from CSV files'
AS
SELECT * FROM read_files(
  '/Volumes/main/default/f1_raw_data/races.csv',
  format => 'csv',
  header => true
)
""")

print("✅ Bronze races table created!")

In [ ]:
# Bronze Table 2: Drivers  
spark.sql("""
CREATE OR REPLACE TABLE main.default.bronze_drivers
USING DELTA
COMMENT 'Bronze layer: Raw F1 driver data from CSV files'
AS
SELECT * FROM read_files(
  '/Volumes/main/default/f1_raw_data/drivers.csv',
  format => 'csv',
  header => true
)
""")

print("✅ Bronze drivers table created!")

In [ ]:
# Bronze Table 3: Results
spark.sql("""
CREATE OR REPLACE TABLE main.default.bronze_results
USING DELTA
COMMENT 'Bronze layer: Raw F1 race results from CSV files'
AS
SELECT * FROM read_files(
  '/Volumes/main/default/f1_raw_data/results.csv',
  format => 'csv',
  header => true
)
""")

print("✅ Bronze results table created!")

## 🥈 Step 6: Silver Layer - Clean and Validated Data

The **Silver layer** contains cleaned, validated, and enriched data. We'll fix data types, handle nulls, and add business logic.

In [ ]:
# Silver Table 1: Clean Races with proper date types
spark.sql("""
CREATE OR REPLACE TABLE main.default.silver_races
USING DELTA
COMMENT 'Silver layer: Cleaned F1 race data with proper data types'
AS
SELECT 
  CAST(raceId as INT) as raceId,
  CAST(year as INT) as year,
  CAST(round as INT) as round,
  CAST(circuitId as INT) as circuitId,
  name as race_name,
  CAST(date as DATE) as race_date,
  time as race_time,
  url as race_url,
  CASE 
    WHEN year < 1980 THEN 'Classic Era'
    WHEN year < 2000 THEN 'Modern Era' 
    WHEN year < 2014 THEN 'V8 Era'
    ELSE 'Hybrid Era'
  END as f1_era,
  current_timestamp() as processed_at
FROM main.default.bronze_races
WHERE year IS NOT NULL AND name IS NOT NULL
""")

print("✅ Silver races table created with cleaned data!")

## 🥇 Step 7: Gold Layer - Analytics-Ready Data

The **Gold layer** contains business-ready aggregated data for analytics and reporting.

In [ ]:
# Gold Table 1: Driver Career Statistics
spark.sql("""
CREATE OR REPLACE TABLE main.default.gold_driver_standings
USING DELTA
COMMENT 'Gold layer: Comprehensive driver career statistics for analytics'
AS
SELECT 
  d.driverId,
  CONCAT(d.forename, ' ', d.surname) as full_name,
  d.nationality,
  COUNT(DISTINCT r.raceId) as total_races,
  SUM(CAST(res.points as DOUBLE)) as total_career_points,
  COUNT(CASE WHEN res.position = '1' THEN 1 END) as wins,
  COUNT(CASE WHEN CAST(res.position as INT) <= 3 THEN 1 END) as podiums,
  ROUND(SUM(CAST(res.points as DOUBLE)) / COUNT(DISTINCT r.raceId), 2) as points_per_race,
  ROUND(COUNT(CASE WHEN res.position = '1' THEN 1 END) * 100.0 / COUNT(DISTINCT r.raceId), 2) as win_percentage,
  MIN(CAST(race.year as INT)) as career_start_year,
  MAX(CAST(race.year as INT)) as career_end_year,
  current_timestamp() as calculated_at
FROM main.default.bronze_drivers d
JOIN main.default.bronze_results res ON CAST(d.driverId as INT) = CAST(res.driverId as INT)
JOIN main.default.bronze_races race ON CAST(res.raceId as INT) = CAST(race.raceId as INT)
WHERE d.forename IS NOT NULL AND d.surname IS NOT NULL
GROUP BY d.driverId, d.forename, d.surname, d.nationality
HAVING COUNT(DISTINCT r.raceId) >= 1
""")

print("✅ Gold driver standings table created with career statistics!")

## ✅ Pipeline Complete! 

**🎉 Congratulations! You've built a complete Formula 1 data lakehouse!**

### What You've Accomplished:
- ✅ **Downloaded real F1 data** from GitHub to Volume storage
- ✅ **Created Delta tables** across Bronze, Silver, and Gold layers
- ✅ **Used modern data patterns** with Volumes and Delta Lake
- ✅ **Implemented data transformations** and business logic
- ✅ **Built analytics-ready** datasets for dashboards

### Your Data Architecture:
```
📁 Volume: main.default.f1_raw_data (3 CSV files)
   ↓
🥉 Bronze: bronze_races, bronze_drivers, bronze_results  
   ↓
🥈 Silver: silver_races (cleaned with F1 eras)
   ↓ 
🥇 Gold: gold_driver_standings (career statistics)
```

In [ ]:
# Final verification - let's check our tables
print("🏁 F1 Data Pipeline Summary")
print("=" * 50)

tables_to_check = [
    'bronze_races', 'bronze_drivers', 'bronze_results',
    'silver_races', 
    'gold_driver_standings'
]

for table in tables_to_check:
    try:
        count = spark.sql(f"SELECT COUNT(*) as count FROM main.default.{table}").collect()[0].count
        layer = table.split('_')[0].upper()
        emoji = {'BRONZE': '🥉', 'SILVER': '🥈', 'GOLD': '🥇'}.get(layer, '📊')
        print(f"{emoji} {table}: {count:,} records")
    except Exception as e:
        print(f"❌ {table}: Error - {str(e)}")

print(f"\n🎯 Pipeline built successfully!")

## 🚀 Next Steps

Your F1 data lakehouse is ready! Here's what to explore next:

### Immediate Next Steps:
1. **➡️ [03_Unity_Catalog_Demo.ipynb](03_Unity_Catalog_Demo.ipynb)** - Explore data lineage and governance
2. **➡️ [04_Job_Creation.ipynb](04_Job_Creation.ipynb)** - Schedule automated data refreshes  
3. **➡️ [07_SQL_Editor.sql](07_SQL_Editor.sql)** - Build analytics queries and visualizations

### Advanced Features:
- **Delta Live Tables** - Managed ETL pipelines
- **AI Agents** - Build F1 Q&A chatbots
- **Dashboards** - Interactive data visualizations
- **Genie** - Natural language queries

**🏎️ Ready to dive deeper into the world of data + AI? Let's go!**